# Data Preparation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyreadstat
import re
import string
from sklearn_pandas import DataFrameMapper
import networkx as nx
import reed
import pickle
from clean import *

pd.options.display.max_columns=100
pd.options.display.max_colwidth=200
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Parameters
Set which waves to base the analysis on, what the minimum age must be to be considered and above what threshold a column is excluded due to missing data.

In [2]:
s,m,e = 'a','q','s' # select which waves to base analysis on
min_start_age = 25 # the minimum age people must as of the starting wave
missing_threshold = 0.90
correlation_threshold = 0.90
redundant_threshold=0.9

## Data Structure

- Part1 contains the combined data from all questionairs asked in a given wave. Each wave is a separate file (eg a s wave 1, be is wave 2, etc. 
- TODO understand the note in HILDA manual that concepts that contained both positive and negative values were split across multiple columns. 

In [3]:
# read the combined file for the starting wave
df1, meta1 = pyreadstat.read_sav(f'../part1/Combined {s}190c.sav') 
n0 = len(df1)
print(f"Number of people in initial wave {n0}")
with open('data/metadata.pkl','wb') as f:
    pickle.dump(meta1,f)

Number of people in initial wave 19914


### Filter people who were already studying

In [4]:
def filter_participants(df1,min_start_age):
    """
    Remove those already studying or below the minimum age in the initial wave.
    """
    n0 = len(df1)
    df = df1.loc[df1[f'{s}hgage'] >= min_start_age].copy()
    print(f"Dropping {n0-len(df)} participants below age {min_start_age}")

    # filter out those already studying

    # If any of the following are > 0, then the respondant was already studying at the beginning of the period
    already_studying_cols = [s+col for col in ['caeft','caept','nlreast','bncsty','bnfsty']]

    already_studying = df[already_studying_cols].sum(axis=1)

    n0 = len(df)
    df = df[already_studying < 1].copy()
    print(f"Dropping {n0-len(df)} participants already studying at period start")
    print(f"Remaining participants:{len(df)}")
    return df

df1 = filter_participants(df1,min_start_age)

Dropping 7359 participants below age 25
Dropping 1216 participants already studying at period start
Remaining participants:11339


### Compute treatment & outcomes
#### Outcomes measures
   - **hours worked** Largly missing. The following variables are perfectly correlated
      - `ajbhru` 57% missing, *E1b Hours per week usually worked in all your jobs*
      - `ajbhruc`,57% missing *DV: Hours per week usually worked in all jobs*
   - **wages (not normalised by hours worked)**
      - `awsfe` wages from all jobs last financial year with imputation from net
      - `awsce` current weekly wages from all jobs with imputation from net
      - Non-imputed versions of both of these exist (replace the final `e` with `g`) but have slightly more missing values
      - `awsfhave` records if people have received income from salary/wages last financial year. We could also use `_esbrd` to tell if people should have a non-zero wage.
      - wage variables have quite a lot of missing data (~33% missing `awsfe` and 30% missing both `awsfe` and `awsfhave`)
      - There are versions of wage variables with imputation of missing data from based on responses from the participant in other waves and responses from similar participants. These are indicated by the suffix `i` (eg `awsfei`, `awscei`). These variables contain no missing data.
   - **employment status (categorical outcome)**
   - **mental health**
   
#### Treatment measures
   - Treatment is based on a change in education qualification between 2001 and 2017
   - There are a number of study related variables that are only recorded on a subset of the waves. 
   - _edq{XXX} variables are recorded every year and count the number of qualifications a person holds in each of a number of categories


In [ ]:
from treatment_outcomes import compute_treatment_vars, compute_outcomes
treatments = compute_treatment_vars(df1, s, m)
outcomes = compute_outcomes(df1, s, e)
treatment_outcomes = pd.merge(treatments,outcomes,on='xwaveid',how='inner')
treatment_outcomes['xwaveid'] = treatment_outcomes['xwaveid'].astype(int)
print("Treatments:",treatments.columns)
print("Outcomes:",outcomes.columns)
print("Updated computation of treatment")

#### Confusion matrix for highest vs count based treatment variable

In [ ]:
from reed import compute_confusion
compute_confusion(treatments['redudl'],treatments['reduhl'],'dl','dh')

## Extract basic variables
Extract a data set corresponding to the original paper we are working to extend - based on the table below;
![image.png](images/original_paper_table.png)

In [ ]:
from treatment_outcomes import simplify_employment
from reed import regex_select

def extract_basic_variables(df):
    # age, sex, education in 2001, employment status in 2001
    basic = df1[['xwaveid','ahgage','ahgsex','aedhigh1','aesdtl']].copy() 

    def simplify_education(v):
        """Simplify down to match categories in paper."""
        if v < 0 or v==10:
            return np.nan # missing
        if v < 3: #(above bachelors)
            return 2
        return v # < year 12:(9), year 12:(8), cert:(5), diploma/adv diploma:(4), bachelors/honours:(3)
    
    # simplify education & employment in line with baseline paper
    basic['aesdtl']=basic['aesdtl'].apply(simplify_employment)
    basic['aedhigh1'] = basic['aedhigh1'].apply(simplify_education)
    
    # bin age
    basic['ahgage'] = pd.cut(basic['ahgage'],bins=[24,34,44,54,120])
    
    # dummy encode
    basic = pd.get_dummies(basic,columns=['ahgage','ahgsex','aedhigh1','aesdtl'],drop_first=True)
    
    # add interactions between gender and other variables
    age_edu_emp = regex_select(basic.columns,['^ahgage_','^aedhigh1_','^aesdtl_'])
    basic = create_interaction_columns(basic,['ahgsex_2.0'],age_edu_emp)
    basic['xwaveid'] = basic['xwaveid'].astype(int)
    return basic

basic = extract_basic_variables(df1)
l0 = len(basic)
basic_with_outcomes = pd.merge(basic,treatment_outcomes,on='xwaveid',how='inner')
l1 = len(basic_with_outcomes)
print(f"Dropped {l0-l1} individuals who are not present in waves {m} and {e} ({100*(l0-l1)/l0:.0f}%)")
basic_with_outcomes.set_index('xwaveid',inplace=True)
basic_with_outcomes.to_csv("basic_variables.csv",index=True)

## Extract Full variable set
Extract a 'kitchen sink' dataset with minimal filtering of features

### Filter out columns based on annotated spreadsheet
Remove columns that have been manually marked as irrelevant or proxies to whether someone is already studying. 

In [ ]:
def read_type_information():
    headers = ['variable','vartype','format','label','long_label','varcat','relevance',"0"]
    type_df = pd.read_csv("HILDAw1vardic.csv",skiprows=4,index_col=None, names=headers)
    type_df['relevance'] = type_df['relevance'].fillna(1).astype(int)
    type_df.loc[type_df['label']=='ACAEPT','relevance'] = -1
    return type_df

def drop_irrelevant_columns_inplace(df, type_df):
    irrelevant = list(type_df.loc[type_df['relevance']<1,'variable'])
    irrelevant.remove('xwaveid')
    df.drop(columns=irrelevant,inplace=True)
    print(f"Dropped {len(irrelevant)} irrelevant columns.")
    return irrelevant

### Fix types
   - encode categorical values & strings as integers (ordinal rather than one-hot)
   - transform dates into days past 01/01/1900

In [ ]:
def fix_types_inplace(df1):
    # Reformat some of the columns
    dates = ["ahhhqivw","ahhcompi","ahhcompf","ahhcomps","ahhidate"]
    string = ['ahhtitle']
    categorical = [
     'acca1',
     'acca2',
     'ahhmgfxd',
     'ahhmgmxd',
     'ahhp1',
     'ahhp2',
     'ahhp3',
     'ahhpgfxd',
     'ahhpgmxd',
     'ahhpno',
     'xwaveid'
    ]

    for c in categorical:
        if c in df1.columns:
            df1[c] = pd.to_numeric(df1[c])

    # turn into days past epoch
    basedate = pd.to_datetime('01/01/1900',format='%d/%m/%Y')    
    for c in dates:
        if c in df1.columns:
            df1[c] = (pd.to_datetime(df1[c],format='%d/%m/%Y',errors='coerce')-basedate).dt.days 

    for c in string:
        df1[c] = df1[c].astype('category').cat.codes
        

In [ ]:
def redundancy_with_nan(series, nbins):
    """
    Compute the redundancy of a series after binning.
    
    Redundancy, in [0,1], measures how close the entropy of the distribution is to the maximum
    entropy given the number of bins. Redundancy is minimized (=0) when the data is uniformly 
    distributed over the bins and maximised (=1) when all the data is in a single bin. 
    """
    if series.nunique() > nbins:
        series = pd.cut(series, nbins, labels=False)
    counts = series.value_counts(dropna=False).values
    if len(counts) == 1:
        return 1
    p = counts/counts.sum()
    entropy = (-p*np.log(p)).sum()
    redundancy = 1 - entropy/np.log(len(counts))
    return redundancy

def drop_redundant_columns_inplace(df,nbins, threshold):
    r = np.zeros(len(df.columns))
    for i,c in enumerate(df.columns):
        r[i] = redundancy_with_nan(df[c],nbins)
    
    exclude = r > threshold
    redundant = list(df.columns[exclude])
    
    r_values = dict(zip(df.columns[~exclude],r[~exclude]))
    
    df.drop(columns=redundant,inplace=True)
    print(f"Dropped {len(redundant)} columns with high redundancy/low entropy")
    return redundant, r_values

In [ ]:
def compute_correlations(df, fillval = 0):
    """Compute the correlations between each pair of variables and return as a DataFrame in long form."""
    c = df.fillna(fillval).corr()
    c1 = []
    c2 = []
    value = []
    for i in range(c.shape[0]):
        for j in range(c.shape[0]):
            if i > j:
                value.append(c.iloc[i, j])
                c1.append(c.index[i])
                c2.append(c.columns[j])
    c = pd.DataFrame({'c1': c1, 'c2': c2, "correlation": value})
    c['abs'] = c['correlation'].abs()
    c.sort_values(['abs'],ascending=False,inplace=True)
    return c

In [ ]:
from collections import defaultdict

def merge_correlated_pairs(df,r_vals, threshold, fillval=0):
    """
    Merges pairs of variables with a correlation coefficient above the threshold.
    
    Returns
    -------
    merged: list[str]
        A list of all the column names that were merged into other columns.
        
    merges: {str:[str]} 
        A dict from column name to all the columns merged with that column.
    """

    cs = compute_correlations(df, fillval)
    
    column_indicies = [
        cs.columns.get_loc('c1'),
        cs.columns.get_loc('c2'),
        cs.columns.get_loc('abs')
    ]

    row = cs.iloc[0,column_indicies]
    c1, c2, t = row
    merges = defaultdict(list)
    while t >= threshold:

        # merge (keep lowest redundancy)
        r1, r2 = r_vals[c1], r_vals[c2]
        if r1 <= r2:
            best, other = c1, c2
        else:
            best, other = c2, c1

        merges[best].append(other)

        # delete all rows involving merged in variable
        drop_index = cs.index[(cs['c1']==other)|(cs['c2']==other)]

        cs.drop(index=drop_index,inplace=True)
        row = cs.iloc[0,column_indicies]
        c1, c2, t = row
    
    merged = []
    for v in merges.values():
        merged.extend(v)
        
    df.drop(columns=merged,inplace=True)
    return merged, merges

### Automatic (very basic) column filtering
   - drop columns where the proportion of data missing is above the maximum threshold
   - drop columns that are constant (zero variance)
   - drop columns that are very tightly correlated (based on correlation threshold) with another column that contains less missing data. 

In [ ]:
def filter_raw_data(df1, missing_threshold=0.99, correlation_threshold=0.99):
    columns_dropped = {} # keep track of why each column was dropped
    
    type_df = read_type_information()
    irrelevant = drop_irrelevant_columns_inplace(df1,type_df)
    add_list_to_dict(irrelevant,columns_dropped,'invalid/irrelevant')

    fix_types_inplace(df1)
    
    constant = drop_constant_columns(df1)
    add_list_to_dict(constant,columns_dropped,'constant')
    
    redundant, r_vals = drop_redundant_columns_inplace(df1, 100, redundant_threshold)
    add_list_to_dict(redundant,columns_dropped,'high-redundancy')

    mostly_missing = drop_mostly_missing_columns(df1, threshold = missing_threshold)
    add_list_to_dict(mostly_missing, columns_dropped, 'mostly-missing')
    
    dropped, merges = merge_correlated_pairs(df1, r_vals, correlation_threshold,fillval=0)
    add_list_to_dict(dropped, columns_dropped, 'merged')
    
    
    print("Processed data, with shape:",df1.shape)
    return df1, columns_dropped, r_vals


# def drop_strongly_correlated_columns(df, columns_dropped,threshold=0.99, inplace=True, fillna=True):
#     """
#     Drop columns that are highly correlated with another column.
    
#     In each strongly connected component, the variable with the least missing data will be the 
#     representative variable kept.
#     """
#     d = df.fillna(-1)
#     c = compute_correlations(d)
#     del d
#     strong = c[c['correlation'].abs() > threshold]
#     print(f"number of strong correlations stronger than {threshold:.2f} is: {len(strong)}")
#     g = nx.Graph()
#     g.add_edges_from(list((strong[['c1','c2']]).itertuples(index=False,name=None)))
#     components = list(nx.connected_components(g))
#     print("Number of connected components in strong correlation graph:",len(components))
    
#     drop = set({})
#     for component in components:
#         least_missing = df[list(component)].isnull().sum(axis=0).idxmin()
#         component.remove(least_missing)
#         drop = drop.union(component)
#         add_list_to_dict(component,columns_dropped,f"merged into {least_missing}")
#     print("Columns dropped due to almost perfect correlation:",len(drop))
#     if inplace:
#         df.drop(columns=drop,inplace=True)
#     return drop, columns_dropped

### Save the data to file
Save the data to file for subsequent model fitting

In [ ]:
def write_data(X, treatment_outcomes,filename):
    l0 = len(X)
    df = pd.merge(X, treatment_outcomes, on=['xwaveid'],how='inner')
    l1 = len(df)
    print(f"Dropped {l0-l1} individuals who are not present in waves {m} and {e} ({100*(l0-l1)/l0:.0f}%)")
    df.set_index('xwaveid',inplace=True)
    df.to_csv(filename,index=True)
    print("Written data to:",filename)
    assert (basic_with_outcomes.index == df.index).all(), "index should be the same across datasets"

X, columns_dropped, r_vals = filter_raw_data(df1, missing_threshold=missing_threshold,correlation_threshold=correlation_threshold)
write_data(X, treatment_outcomes, "all_vars.csv")

## Feature selection using current wage as labels

This is based on the premise that the features that are predictive of the change in wage due to returning to education are likely to be the same set of features that are important for predicting the initial wage. This allows us to do supervised feature selection without worrying about over-fitting to the data, as we are not using the final labels we are training against. 

In [ ]:
from dimensionality import effective_rank
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression
from sklearn.preprocessing import StandardScaler

In [ ]:
target = 'awscei'
if 'xwaveid' in X.columns:
    X.set_index('xwaveid',inplace=True)
y = X[target].fillna(0).values

features = regex_select(X.columns, 'wscei', exclude=True)
Xs = StandardScaler().fit_transform(X[features].fillna(0))
Xs = pd.DataFrame(Xs, columns=features)

### Univariate features selectiong: Select k-best

Takes no account of correlation between features, results in a matrix with a low effective rank (compared with it's actual rank). 

In [ ]:
selector = SelectKBest(f_regression, k=100)
selector.fit(Xs,y)
effective_rank(Xs[selector.get_feature_names_out()])

### Multivariate-feature selection: Lasso

In [ ]:
# see how many features we get as a function of regularisation strength. 
# we could use prediction accuracy as a rough estimate of how many features to use
from sklearn.linear_model import Lasso, LassoCV
from sklearn.feature_selection import SelectFromModel

alphas = np.logspace(-1,1,20)
n_selected = []
for alpha in alphas:
    ls = Lasso(alpha=alpha)
    selector = SelectFromModel(ls)
    selector.fit(Xs,y)
    n_selected.append(len(selector.get_feature_names_out()))

plt.semilogx(alphas, n_selected)
plt.xlabel("alpha")
plt.ylabel("number of features selected")

In [ ]:
# select features with Lasso and write data to file. 
ls = Lasso(alpha=3)
selector = SelectFromModel(ls)
selector.fit(Xs,y)
f_selected = selector.get_feature_names_out()
print("number of features selected:",len(f_selected))
print("effective rank:",effective_rank(Xs[f_selected]))

write_data(X[f_selected].reset_index(), treatment_outcomes, "all_lasso_selected.csv")

## Unsupervised feature selection

Greedy selection of features to optimise effective rank.

In [ ]:
from cinspect.dimension import greedy_feature_selection, effective_rank

features_to_select = 100
Xs = StandardScaler().fit_transform(X.fillna(0))
selected, vals = greedy_feature_selection(
    Xs, effective_rank, 
    initial_col=4, num_to_select=features_to_select
)

selected_cols = [X.columns[i] for i in selected]
X_su = X[selected_cols]
print("number of features selected:",len(selected_cols))
print("effective rank:",effective_rank(Xs[selected]))
write_data(X_su.reset_index(), treatment_outcomes, "all_unsupervised_selected.csv")